https://www.kaggle.com/datasets/narendrageek/can-generate-automatic-commentary-for-ipl-cricket?select=IPL_SCHEDULE_2008_2020.csv

# LIBRARIES 

In [2]:
import re
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import requests
from bs4 import BeautifulSoup

In [15]:
url = 'https://www.cricbuzz.com/live-cricket-scorecard/10545/kxip-vs-dc-13th-match-indian-premier-league-2008' 
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract scorecard data
scorecard = soup.find_all('div', class_='cb-col cb-col-100 cb-ltst-wgt-hdr')

temp = ''
for section in scorecard:
    # print(section.get_text(separator='\n'))
    temp += section.get_text(separator='\n')
temp = temp.replace("\n", "")

In [16]:
temp

'  Delhi Daredevils Innings 158-8 (20 Ov)   Batter  R B 4s 6s SR      Yo Mahesh    not out  6 5 0 0 120.00        Farveez Maharoof    run out (Jayawardene/Sangakkara)  18 12 2 0 150.00        Rajat Bhatia    b Pathan  17 15 1 1 113.33        Manoj Tiwary    c Yuvraj Singh b Singh  39 34 6 0 114.71        Dinesh Karthik    b Yuvraj Singh  24 14 2 1 171.43        Shoaib Malik    c Lee b Singh  19 17 3 0 111.76        Virender Sehwag    lbw b Pathan  6 6 1 0 100.00        Shikhar Dhawan    c Sangakkara b Lee  0 3 0 0 0.00        Gautam Gambhir    c Yuvraj Singh b Singh  18 14 3 0 128.57      Extras  11  \xa0(b 0, lb 5, w 6, nb 0, p 0)   Total  158   \xa0(8 wkts, 20 Ov)     Bowler O M R W NB WD ECO     Piyush Chawla   4 0 29 0 0 1 7.25        Vikram Singh   4 0 29 3 0 0 7.25        S Sreesanth   2 0 25 0 0 0 12.50        Irfan Pathan   4 0 26 2 0 3 6.50        Brett Lee   4 0 33 1 0 0 8.25        Yuvraj Singh   2 0 11 1 0 1 5.50       Kings XI Punjab Innings 162-6 (19.3 Ov)   Batter  R B 4

In [50]:
# Extract match_id and year from the URL
match_id = int(re.search(r'/(\d+)/', url).group(1))
year = int(re.search(r'(\d+)$', url).group(1))

# Extract team names
teams = re.findall(r'([A-Za-z\s]+) Innings', temp)
teams = [team.strip() for team in teams]

# Split the scorecard into innings
innings = temp.split(f"{teams[1]} Innings")
innings_data = [innings[0], innings[1]]

# Create empty lists for storing data
batters_data = []
bowlers_data = []
extras_data = []

# Regex patterns
# batters_regex = r'([A-Za-z\s]+)\s+(not out|run out|c [A-Za-z\s]+ b [A-Za-z\s]+|st [A-Za-z\s]+ b [A-Za-z\s]+|b [A-Za-z\s]+|lbw b [A-Za-z\s]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+([\d.]+)'
batters_regex = r'([A-Za-z\s]+)\s+(not out|run out\(.+?\)|c & b [A-Za-z\s]+|c [A-Za-z\s]+ b [A-Za-z\s]+|b [A-Za-z\s]+|lbw b [A-Za-z\s]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+([\d.]+)'
bowlers_regex = r'([A-Za-z\s]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+([\d.]+)'
extras_pattern = r'Extras\s+(\d+)\s+\(b (\d+), lb (\d+), w (\d+), nb (\d+), p (\d+)\)'
total_pattern = r'Total\s+(\d+)\s+\((\d+)\s+wkts?,\s+(\d+(\.\d+)?)\s+Ov\)'

# Loop through the innings and extract relevant data
for i, inning in enumerate(innings_data):
    inning = re.sub(r'^.*?R B 4s 6s SR\s*', '', inning)
    # print(i,inning)
    batting_team = teams[i]
    bowling_team = teams[1 - i]

    # Extract batters data
    batters = re.findall(batters_regex, inning)
    for batter in batters:
        # dismissal_type = batter[1]
        batter_name_with_dismissal = f"{batter[0].strip()} {batter[1].strip()}"
        batters_data.append({
            'Match ID': match_id,
            'Year': year,
            'Team': batting_team,
            'Batter': batter_name_with_dismissal,
            # 'Batter': batter[0],
            # 'Dismissal': dismissal_type,
            'R': int(batter[2]),
            'B': int(batter[3]),
            '4s': int(batter[4]),
            '6s': int(batter[5]),
            'SR': float(batter[6]),
        })


    # Extract extras+total data
    extras_match = re.search(extras_pattern, inning)
    if extras_match:
        # Store the extracted data into a dictionary
        extras_data.append({
            'Match ID': match_id,
            'Year': year,
            'Team': bowling_team,
            'Total_extras': int(extras_match.group(1)),
            'Byes': int(extras_match.group(2)),
            'Legbyes': int(extras_match.group(3)),
            'Wides': int(extras_match.group(4)),
            'Noballs': int(extras_match.group(5)),
            'Penalty_runs': int(extras_match.group(6)),
        })
        # print("Extras data fetched sucessfully!!!")
    else:
        print("Extras and Total data not found.")
   
    total_match = re.search(total_pattern, inning)

    if total_match:
        extras_data[i]['Total_Score'] = int(total_match.group(1))
        extras_data[i]['Total_wkts'] = int(total_match.group(2))
        extras_data[i]['Total_Overs'] = float(total_match.group(3))
    else:
        print("Total data not found.")
    

    inning = re.sub(r'^.*?Bowler O M R W NB WD ECO\s*', '', inning)
    # Extract bowlers data
    bowlers = re.findall(bowlers_regex, inning)
    for bowler in bowlers:
        bowlers_data.append({
            'Match ID': match_id,
            'Year': year,
            'Team': bowling_team,
            'Bowler': bowler[0].strip(),
            'O': int(bowler[1]),
            'M': int(bowler[2]),
            'R': int(bowler[3]),
            'W': int(bowler[4]),
            'NB': int(bowler[5]),
            'WD': int(bowler[6]),
            'ECO': float(bowler[7]),
        })


extras_data[1]['Total_Score'], extras_data[0]['Total_Score'] = extras_data[0]['Total_Score'], extras_data[1]['Total_Score']
extras_data[1]['Total_Overs'], extras_data[0]['Total_Overs'] = extras_data[0]['Total_Overs'], extras_data[1]['Total_Overs']

batters_df = pd.DataFrame(batters_data)
bowlers_df = pd.DataFrame(bowlers_data)
extras_df = pd.DataFrame(extras_data)

# print("Batters DataFrame:")
# print(batters_df)
# print("\nBowlers DataFrame:")
# print(bowlers_df)
# print("\Extras DataFrame:")
# print(extras_df)

file_name = 'cricket_scorecards.xlsx'
with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    start_row = writer.sheets['Batters'].max_row  # Get the last row of the sheet
    batters_df.to_excel(writer, sheet_name='Batters', startrow=start_row, header=False, index=False)
    start_row = writer.sheets['Bowlers'].max_row
    bowlers_df.to_excel(writer, sheet_name='Bowlers', startrow=start_row, header=False, index=False)
    start_row = writer.sheets['Extras'].max_row
    extras_df.to_excel(writer, sheet_name='Extras', startrow=start_row, header=False, index=False)

print(f"DataFrames appended to '{file_name}' successfully!")


DataFrames appended to 'cricket_scorecards.xlsx' successfully!
